<a href="https://colab.research.google.com/github/spendnetwork/charts/blob/master/amount_of_transactions_by_ministry_of_defence_chart_opensourced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import altair as alt
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [57]:
url = 'https://raw.githubusercontent.com/spendnetwork/charts/master/ministry_of_defence_spend.csv'
df = pd.read_csv(url)
print(df)

                          trans_id        buyer_id  ...        date        amount
0      MOD017_MOD_gov_2017_03_1879  MOD017_MOD_gov  ...  2017-03-01  4.500000e+04
1      MOD017_MOD_gov_2019_01_2562  MOD017_MOD_gov  ...  2019-01-01  5.005894e+08
2      MOD017_MOD_gov_2019_04_1193  MOD017_MOD_gov  ...  2019-04-01  4.200000e+04
3      MOD017_MOD_gov_2018_05_1124  MOD017_MOD_gov  ...  2018-05-01  3.854000e+04
4       MOD017_MOD_gov_2018_10_840  MOD017_MOD_gov  ...  2018-10-01  6.132240e+04
...                            ...             ...  ...         ...           ...
94186   MOD017_MOD_gov_2017_09_329  MOD017_MOD_gov  ...  2017-09-01  2.660568e+05
94187   MOD017_MOD_gov_2017_10_848  MOD017_MOD_gov  ...  2017-10-01  4.583300e+04
94188  MOD017_MOD_gov_2017_11_1012  MOD017_MOD_gov  ...  2017-11-01  4.496420e+04
94189   MOD017_MOD_gov_2017_01_199  MOD017_MOD_gov  ...  2017-01-01  2.142146e+05
94190   MOD017_MOD_gov_2017_11_591  MOD017_MOD_gov  ...  2017-11-01  1.123472e+05

[94191 rows x 1

In [0]:
df['date']=pd.to_datetime(df['date']).dt.strftime('%m/%d/%Y')

In [0]:
df1=(df.groupby('date').agg({'trans_id': ['count'], 'amount': 'sum'}))
#df1.head()

In [0]:
df1.columns = df1.columns.map('_'.join)
df1 = df1.reset_index()
#df1.head()

In [0]:
#df1['releasedate']=pd.to_datetime(df1['releasedate'].dt.strftime('%m/%d/%Y'))
#df1['releasedate']=pd.to_datetime(df1['releasedate']).dt.strftime('%m/%d/%Y') 
#df1.head()

In [0]:
def rolling_twelve_month_trend(df, colname, agg_type):        
        df2=df1['date'] = pd.to_datetime(df1['date'])
        p2mask = (df1['date'] > datetime.today() - relativedelta(years=1)) & (df1['date'] <= datetime.today())
        p1mask = (df1['date'] > datetime.today() - relativedelta(years=2)) & (
                    df1['date'] <= datetime.today() - relativedelta(years=1))
        p2 = df.loc[p2mask]
        p1 = df.loc[p1mask]

        perc = None
        if agg_type == 'sum':
            perc = float(p2[colname].sum() - p1[colname].sum()) / abs(p1[colname].sum()) * 100
        elif agg_type == 'mean':
            perc = float(p2[colname].mean() - p1[colname].mean()) / abs(p1[colname].mean()) * 100
        return perc
  
def which_caret(stat_value):
        stat_value = int(stat_value)
        ## returning html that deploys Font Awesome icons, can be changed according to icon set in use
        if stat_value > 0:
            caret = u"\u25B2"
        elif stat_value < 0:
            caret = u"\u25BC"
        else:
            caret = u"\u003D"
        return caret

def inc_dec(stat_value):
        stat_value = int(stat_value)
        ## returning html that deploys Font Awesome icons, can be changed according to icon set in use
        if stat_value > 0:
            change = 'increase'
        elif stat_value < 0:
            change = 'decrease'
        else:
            change = 'change'
        return change

In [63]:
perc = rolling_twelve_month_trend(df1, 'amount_sum', 'sum')
caret = which_caret(perc)
change = inc_dec(perc)

caret+str(perc)+change

'▼-31.22337500452054decrease'

In [64]:
        chart = (
            alt.Chart(df1, width=1200, height=250, title="Total value of spend by Ministry of Defence")
                .mark_bar(color='#91D739')
                .encode(
                alt.X(
                    "yearmonth(date):O",
                    ## xAxis title must be "Source: https://openopps.com"
                    axis=alt.Axis(title="Source: https://openopps.com", grid=False, 
                labelFont='Helvetica', 
                labelFontSize=12, 
                labelAngle=0, 
                titleAlign='left',
                titleAnchor='start',
                titleFont='Helvetica',
                titleFontSize= 11,
                titleFontWeight= 300,
                titleX= -0,
                titleY= 30)
                ),
                alt.Y("sum(amount_sum):Q",
                      axis=alt.Axis(title=(caret +' '+ str(int(perc))+'% ' + change + ' this year in comparison with the previous year.' ),
                                    format=".2s",
                                    titleAlign= 'left',
                                    titleAnchor='start',
                                    titleAngle= 0,
                                    titleFont= 'Helvetica',
                                    titleFontSize= 12,
                                    titleFontWeight= 400,
                                    titleX= -0,
                                    titleY= -30,),
                      ),
                tooltip=[
                    alt.Tooltip('sum(amount_sum):Q', title="Total value", format="$.4r"),
                    alt.Tooltip('sum(trans_id_count):Q', title="Total transactions")
                ],
                  # suppress the legend for now.
            )
        )

        text = (
            chart.mark_text(align="center", baseline="bottom")
                .encode(text=alt.Text('sum(amount_sum):N', format="$.3s"))
        )

        (chart+text).configure_view(
          strokeWidth=0
        ).configure_title(fontSize=18, anchor='start',
                color= "#91D739", font='Helvetica', fontWeight=400)


alt.LayerChart(...)